<a href="https://colab.research.google.com/github/ZynTran/Yolo/blob/main/CCCDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# --- Cài thêm tesseract và dữ liệu tiếng Việt ---
!apt-get install tesseract-ocr -y
!apt-get install tesseract-ocr-vie -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-vie
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 417 kB of archives.
After this operation, 546 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-vie all 1:4.00~git30-7274cfa-1.1 [417 kB]
Fetched 417 kB in 1s (532 kB/s)
Selecting previously unselected package tesseract-ocr-vie.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-vie_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-vie (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-vie 

In [14]:
pip install streamlit pytesseract pandas openpyxl pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [15]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 5.6 MB/s eta 0:00:00


In [36]:
import easyocr
import pandas as pd
import re
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

folder_path = '/content/cccd'
reader = easyocr.Reader(['vi', 'en'])
def correct_common_errors(text):
    corrections = {
        'do\n': 'Tự do\n',
        'Hanh phúc': 'Hạnh phúc',
        'Viêt Nam': 'Việt Nam',
        'Dale of expiry': 'Date of expiry',
        'Cógiá trị': 'Có giá trị',
    }
    for wrong, right in corrections.items():
        text = text.replace(wrong, right)
    return text
def clean_text(text):
    stop_keywords = ['Có giá trị', 'Giá trị', 'Valid until', 'Ngày', 'Date']
    for keyword in stop_keywords:
        if keyword in text:
            return text.split(keyword)[0].strip()
    date_pattern = r'\d{1,2}/\d{1,2}/\d{2,4}'
    match = re.search(date_pattern, text)
    if match:
        return text[:match.start()].strip()
    return text.strip()

def extract_info(text):
    info = {
        'Họ và tên/Full name': '',
        'Ngày sinh/Date of birth': '',
        'Giới tính/Sex': '',
        'Quốc tịch/Nationality': '',
        'Quê quán/Place of origin': '',
        'Nơi thường trú/Place of residence': '',
        'Số CCCD/No': '',
        'Ngày hết hạn/Date of expiry': ''
    }

    lines = [line.strip() for line in text.split('\n') if line.strip()]
    for line in lines:
        match = re.search(r'\b\d{12}\b', line)
        if match:
            info['Số CCCD/No'] = match.group()
            break
    for idx, line in enumerate(lines):
        if 'Họ và tên' in line or 'Full name' in line:
            for j in range(1, 3):
                if idx + j < len(lines):
                    possible_name = lines[idx + j]
                    if len(possible_name.split()) >= 2 and not re.search(r'\d', possible_name):
                        info['Họ và tên/Full name'] = possible_name
                        break
            if info['Họ và tên/Full name']:
                break
    for line in lines:
        dob = re.search(r'\d{2}/\d{2}/\d{4}', line)
        if dob:
            info['Ngày sinh/Date of birth'] = dob.group()
            break
    for line in lines:
        if re.search(r'gi[oí]i\s*t[ií]nh', line, re.IGNORECASE):
            if re.search(r'n[ưu]', line, re.IGNORECASE):
                info['Giới tính/Sex'] = 'Nữ'
            elif re.search(r'nam', line, re.IGNORECASE):
                info['Giới tính/Sex'] = 'Nam'
            break
    if not info['Giới tính/Sex']:
        if re.search(r'\bNữ\b', text, re.IGNORECASE):
            info['Giới tính/Sex'] = 'Nữ'
        elif re.search(r'\bNam\b', text, re.IGNORECASE):
            info['Giới tính/Sex'] = 'Nam'
    if re.search(r'Vi[ệe]t\s*Nam', text, re.IGNORECASE):
        info['Quốc tịch/Nationality'] = 'Việt Nam'
    place_origin = []
    in_origin = False
    for line in lines:
        if 'Quê quán' in line:
            in_origin = True
            continue
        if 'Nơi thường trú' in line or 'Place of residence' in line:
            break
        if in_origin:
            place_origin.append(line)
    info['Quê quán/Place of origin'] = ', '.join(place_origin)
    residence_lines = []
    start_idx = -1
    for i, line in enumerate(lines):
        if re.search(r'Nơi thường trú|Place of residence', line, re.IGNORECASE):
            start_idx = i
            break
    if start_idx != -1:
        for j in range(start_idx + 1, min(len(lines), start_idx + 10)):
            clean_line = lines[j].strip()
            if clean_line in ["Có", "Tự", '"', "giá", ""] or len(clean_line) < 4:
                continue
            if re.search(r'\b(Ấp|xã|phường|thôn|tổ|làng|quận|huyện|Lộc|Bình|Ninh|Phước|Thiện|Đồng|Hòa)\b', clean_line, re.IGNORECASE):
                residence_lines.append(clean_line)

        full_residence = ', '.join(residence_lines)
        full_residence = re.sub(r'\s{2,}', ' ', full_residence).strip(' ,')
        info['Nơi thường trú/Place of residence'] = full_residence
    for i, line in enumerate(lines):
        if any(x in line for x in ['Có giá trị đến', 'Date of expiry', 'Valid until', 'trị đến']):
            date_match = re.search(r'\d{2}/\d{2}/\d{4}', line)
            if date_match:
                info['Ngày hết hạn/Date of expiry'] = date_match.group()
            elif i + 1 < len(lines):
                date_match = re.search(r'\d{2}/\d{2}/\d{4}', lines[i + 1])
                if date_match:
                    info['Ngày hết hạn/Date of expiry'] = date_match.group()
            break
    return info
all_info = []
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(folder_path, filename)
        print(f"\n📄 Đang xử lý: {filename}")

        results = reader.readtext(image_path, detail=0)
        full_text = "\n".join(results)
        full_text = correct_common_errors(full_text)

        info = extract_info(full_text)
        info['Tên file'] = filename
        all_info.append(info)

df = pd.DataFrame(all_info)
output_file = 'thong_tin_nhieu_cccd.xlsx'
df.to_excel(output_file, index=False)
print(f"\n Đã lưu toàn bộ kết quả vào: {output_file}")


📄 Đang xử lý: cccd3.jpg

📄 Đang xử lý: cccd4.jpg

📄 Đang xử lý: cccd1.jpg

📄 Đang xử lý: cccd2.jpg

📄 Đang xử lý: cccd6.jpg

📄 Đang xử lý: cccd5.jpg

 Đã lưu toàn bộ kết quả vào: thong_tin_nhieu_cccd.xlsx
